In [1]:
from utils import *

In [2]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [5]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [6]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [7]:
from azureml.data import OutputFileDatasetConfig

data_out = OutputFileDatasetConfig(name="predictions", destination=(datastore, 'titanic/predictions'))
data_out = data_out.read_delimited_files().register_on_complete('titanic.pred')

In [8]:
from azureml.pipeline.steps import PythonScriptStep

step_1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [9]:
from azureml.pipeline.core.graph import PipelineParameter

lr_param = PipelineParameter(name="lr_arg", default_value=0.01)

In [10]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='Training',
                          script_name="train_output.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test,
                              "--output", data_out,
                              "--learning-rate", lr_param],
                          inputs=[data_train, data_test],
                          outputs=[data_out],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

In [11]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_1, step_2])

In [12]:
pipeline.validate()

Step Training is ready to be created [4efcae3f]


[]

In [15]:
service = pipeline.publish(name="AzureML Published Pipeline", version="1.0")

print(service)
service_id = service.id
service_endpoint = service.endpoint

Pipeline(Name: AzureML Published Pipeline,
Id: 8ce894d6-5aea-406e-9669-d0edbe6eb19e,
Status: Active,
Endpoint: https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourceGroups/packt/providers/Microsoft.MachineLearningServices/workspaces/mldemows/PipelineRuns/PipelineSubmit/8ce894d6-5aea-406e-9669-d0edbe6eb19e)


In [16]:
from azureml.pipeline.core import PipelineEndpoint

endpoint = PipelineEndpoint.publish(ws, pipeline=service, name="AzureML Published Pipeline Endpoint", description="Mastering Azure Machine Learning")

print(endpoint)
service_id = endpoint.id
service_endpoint = endpoint.endpoint

PipelineEndpoint(Name: AzureML Published Pipeline Endpoint,
Id: 89692b17-a436-498c-8aa9-8b60fa58c547,
Description: Mastering Azure Machine Learning,
Pipelines: [('0', Pipeline(Name: AzureML Published Pipeline,
Id: 8ce894d6-5aea-406e-9669-d0edbe6eb19e,
Status: Active,
Endpoint: https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourceGroups/packt/providers/Microsoft.MachineLearningServices/workspaces/mldemows/PipelineRuns/PipelineSubmit/8ce894d6-5aea-406e-9669-d0edbe6eb19e))])


In [17]:
from azureml.core.authentication import AzureCliAuthentication

cli_auth = AzureCliAuthentication()
aad_token = cli_auth.get_authentication_header()

In [18]:
import requests

request = {
    "ExperimentName": "azureml-pipeline-trigger",
    "ParameterAssignments": {
        "lr_arg": 0.05
    }
}

response = requests.post(service_endpoint, headers=aad_token, json=request)
print(response.json)

<bound method Response.json of <Response [200]>>


In [ ]:
# service.disable()
# endpoint.disable()